In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
from tensorflow import keras
import tensorflow as tf
import warnings
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectPercentile
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user
import datetime
import os

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from tensorflow import keras
from scipy.stats.mstats import gmean
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
    

In [206]:
trn = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding='cp949')
tst = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding='cp949')
y = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949')
IDtest = tst.custid.unique()
data = pd.concat([trn, tst], axis=0)

In [207]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1040039 entries, 0 to 414954
Data columns (total 19 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   custid           1040039 non-null  int64 
 1   sales_month      1040039 non-null  int64 
 2   sales_day        1040039 non-null  int64 
 3   sales_dayofweek  1040039 non-null  object
 4   sales_time       1040039 non-null  int64 
 5   str_nm           1040039 non-null  object
 6   goodcd           1040039 non-null  int64 
 7   brd_nm           1040039 non-null  object
 8   corner_nm        1040039 non-null  object
 9   pc_nm            1040039 non-null  object
 10  part_nm          1040039 non-null  object
 11  team_nm          1040039 non-null  object
 12  buyer_nm         1040039 non-null  object
 13  import_flg       1040039 non-null  int64 
 14  tot_amt          1040039 non-null  int64 
 15  dis_amt          1040039 non-null  int64 
 16  net_amt          1040039 non-null  in

날짜 데이터

In [208]:
data['sales_year']=data['sales_month'].apply(lambda x: 2018 if x >12 else 2017)

In [209]:
# 월 정리
data['sales_month']=data['sales_month'].apply(lambda x: x-12 if x >12 else x)

In [210]:
# 주말
data['weekend'] = data['sales_dayofweek'].apply(lambda x: 1 if (x=='토' or x=='일') else 0)

In [211]:
data['sales_year']=data['sales_year'].astype('str')
data['sales_month']=data['sales_month'].astype('str')
data['sales_day']=data['sales_day'].astype('str')

In [212]:
data['date_time']=data['sales_year']+'-'+data['sales_month']+'-'+data['sales_day']

In [213]:
data['sales_year']=data['sales_year'].astype('int')
data['sales_month']=data['sales_month'].astype('int')
data['sales_day']=data['sales_day'].astype('int')

In [214]:
data['date_time'] = pd.to_datetime(data['date_time'])

In [215]:
a=data.groupby(['custid'])['date_time'].diff()

In [216]:
a=a.reset_index().set_index(data.custid)

In [217]:
a=a.reset_index()

In [218]:
a.set_index('index',inplace=True)

In [219]:
data['date_time_diff']=a.date_time

In [220]:
dtdt=data.groupby(['custid','date_time'])['date_time_diff'].sum().reset_index()

In [221]:
dtdt.date_time_diff=dtdt.date_time_diff.astype('str')

In [222]:
dtdt.date_time_diff=dtdt.date_time_diff.apply(lambda x: x.split(' ')[0])

In [223]:
dtdt.date_time_diff=dtdt.date_time_diff.astype('int')

In [224]:
dtdt_mean=dtdt.groupby('custid')['date_time_diff'].mean().reset_index()

# 쇼핑시간

In [225]:
dtdt_mean

,custid,date_time_diff
0,0,41.142857
1,2,18.428571
2,3,25.461538
3,4,1.500000
4,5,15.333333
...,...,...
35962,49988,28.000000
35963,49990,0.000000
35964,49992,142.000000
35965,49993,0.000000


In [226]:
ab=data.groupby(['custid','sales_day','sales_month'])['sales_time'].agg([('sales_time_min', 'min')]).reset_index()

In [227]:
abc=data.groupby(['custid','sales_day','sales_month'])['sales_time'].agg([('sales_time_max', 'max')]).reset_index()

In [228]:
ab['time_min']=ab['sales_time_min'].apply(lambda x: int(str(x)[:2]) if len(str(x)) == 4 else int(str(x)[:1]))
ab['minute_min']=ab['sales_time_min'].apply(lambda x: int(str(x)[2:]) if len(str(x)) == 4 else int(str(x)[1:]))

In [229]:
ab['time_max']=abc['sales_time_max'].apply(lambda x: int(str(x)[:2]) if len(str(x)) == 4 else int(str(x)[:1]))
ab['minute_max']=abc['sales_time_max'].apply(lambda x: int(str(x)[2:]) if len(str(x)) == 4 else int(str(x)[1:]))

In [230]:
ab['shop_time']=(ab['time_max']-ab['time_min'])*60+(ab['minute_max']-ab['minute_min'])

In [231]:
shop_time=ab.groupby('custid')['shop_time'].mean().reset_index()

In [232]:
# sales time - 아침/ 낮/ 밤 구매 나누기
data['sales_time'] = data['sales_time'].apply(lambda x: int(str(x)[:2]) if len(str(x)) == 4 else int(str(x)[:1]))
data['sales_time_cat'] = data['sales_time'].apply(lambda x: 'mo_sales' if (9 <= x <= 12) else ('af_sales' if (13 <= x <= 18) else 'knt_sales'))

In [233]:
# dis_amt - 할인율 특성 추가
data['dis_rate'] = data['dis_amt']/data['tot_amt']

In [234]:
# 퇴근전/후 구매 빈도, 비율
def get_off_work(x):
    if 9<=x<=17:
        return('bf_work')
    else:
        return('af_work')

In [235]:
data['sales_type'] = data['sales_time'].map(get_off_work)

시간별 구매 횟수

In [236]:
freq_per_time = pd.pivot_table(index='custid',columns='sales_time',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt>0")[['custid','sales_time','tot_amt']]).reset_index()

In [237]:
sales_type = pd.pivot_table(index='custid',columns='sales_type',values='tot_amt',aggfunc=np.size, fill_value=0,data = data.query('tot_amt>0')[['custid','sales_type','tot_amt']]).reset_index()

In [238]:
freq_per_time.columns=['custid', '1_stf', '9_stf', "10_stf", '11_stf', '12_stf', '13_stf', '14_stf', '15_stf', '16_stf', '17_stf', '18_stf', '19_stf', '20_stf', '21_stf', '22_stf']

In [239]:
features = pd.merge(freq_per_time, sales_type, on = 'custid')

In [240]:
features = pd.merge(features, shop_time, on = 'custid')

In [241]:
features = pd.merge(features, dtdt_mean, on = 'custid')

In [242]:
def floor_transfer_a(x) :
    return  {
        "무역점" : 0,
        "본점" : 1,
        "신촌점" : 0,
        "천호점" : 1
    }.get(x)

def floor_transfer_m(x) :  
    return  {
        "무역점" : 1/10*13,
        "본점" : 1/6*13,
        "신촌점" : 1/11*13,
        "천호점" : 1
    }.get(x)
data['floor'] = (data.str_nm.apply(floor_transfer_a))*data.str_nm.apply(floor_transfer_m).astype(int)

In [243]:
features = pd.merge(features, data.groupby('custid')['floor'].mean().reset_index(), on = 'custid')

In [244]:
features['af_work_rate'] = features['af_work']/(features['af_work'] + features['bf_work'])
features['bf_work_rate'] = features['bf_work']/(features['af_work'] + features['bf_work'])

In [245]:
# 아침구매/ 낮구매/ 밤구매
freq_sales_cat = pd.pivot_table(index='custid',columns='sales_time_cat',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt>0")[['custid','sales_time_cat','tot_amt']]).reset_index()
features = pd.merge(features, freq_sales_cat,on='custid')

In [246]:
refund_freq=data.query('tot_amt<0').groupby('custid')['sales_day'].count().reset_index()
features = pd.merge(features, refund_freq, left_on = 'custid',right_on='custid',how='left')
features.rename(columns = {'sales_day': 'refund_freq'},inplace=True)
features.fillna(0,inplace=True)

In [247]:
sales_freq=data.query('tot_amt>0').groupby('custid')['sales_day'].count().reset_index()
features = pd.merge(features, sales_freq, left_on = 'custid',right_on='custid',how='left')
features.rename(columns = {'sales_day': 'sales_freq'},inplace=True)

In [248]:
# 구매 빈도 특성 추가
all_freq=data.groupby('custid')['sales_day'].count().reset_index()
features = pd.merge(features, all_freq,  left_on = 'custid',right_on='custid',how='left')
features.rename(columns = {'sales_day': 'all_freq'},inplace=True)

In [249]:
features['refund_sales_rate']=features['refund_freq']/features['sales_freq']
features['refund_all_rate'] = features['refund_freq']/features['all_freq']

방문한 날짜의 갯수

In [250]:
data_aaa=data.groupby(['custid','sales_month','sales_day'])['sales_dayofweek'].count().reset_index()

In [251]:
data_aaa.sales_dayofweek=1

In [252]:
data_aaa=data_aaa.groupby('custid')['sales_dayofweek'].sum().reset_index()

In [253]:
features = pd.merge(features, data_aaa, on = 'custid')

In [254]:
features['sales_freq_mean']=features['sales_freq']/features['sales_dayofweek']

평일에 방문하는 평균 시간

In [255]:
vuddlf=data[data.weekend==0].groupby('custid')['sales_time'].mean().reset_index()

In [256]:
features = pd.merge(features, vuddlf, how='left',on = 'custid')

In [257]:
wkdtjdgus=data[data.weekend==0].groupby('custid')['sales_time'].agg([('sales_time_mod', lambda x: x.value_counts().index[0])]).reset_index()

In [258]:
features = pd.merge(features, wkdtjdgus, how='left',on = 'custid')

주말에 방문하는 평균 시간

In [259]:
wnakf=data[data.weekend==1].groupby('custid')['sales_time'].mean().reset_index()

In [260]:
features = pd.merge(features, wnakf, how='left',on = 'custid')

In [261]:
wkdtjdgus=data[data.weekend==1].groupby('custid')['sales_time'].agg([('sales_time_mod', lambda x: x.value_counts().index[0])]).reset_index()

In [262]:
features = pd.merge(features, wkdtjdgus, how='left',on = 'custid')

In [263]:
# 수입상품 관련 열
sales_import=data.groupby('custid')['import_flg'].agg('sum').reset_index()
features = pd.merge(features, sales_import, on = 'custid')

features.rename(columns = {'import_flg': 'sales_import'},inplace=True)

In [264]:
# 수입상품 구매 빈도 추가
features['import_rate']=features['sales_import']/features['sales_freq']

In [265]:
# 수입상품 환불 빈도 추가
features['import_refund_rate']=features['sales_import']/(features['refund_freq']+1)

In [266]:
# 계절별 구매 빈도 추가
def f1(x):
    if 3 <= x <= 5 :
        return('spring_sales')
    elif 6 <= x <= 8 :
        return('summer_sales')
    elif 9 <= x <= 11 :
        return('fall_sales')
    else :
        return('winter_sales')    

In [267]:
data['season_sales']=data['sales_month'].apply(f1)

In [268]:
season = pd.pivot_table(data, index='custid', columns='season_sales', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features = pd.merge(features, season, how='left',on = 'custid')

In [269]:
#계절별 구매만 빈도
season_sales_freq = pd.pivot_table(index='custid',columns='season_sales',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt>0")[['custid','season_sales','tot_amt']]).reset_index()
features = pd.merge(features, season_sales_freq, how='left',on = 'custid')

In [270]:
#계절별 환불만 빈도
season_refund_freq = pd.pivot_table(index='custid',columns='season_sales',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt<0")[['custid','season_sales','tot_amt']]).reset_index()
features = pd.merge(features, season_refund_freq, how='left',on = 'custid')
features.fillna(0,inplace=True)

In [271]:
features['sales_fall_rate']=features.fall_sales_x/(features.fall_sales_x+features.spring_sales_x+features.summer_sales_x+features.winter_sales_x)

In [272]:
features['sales_spring_rate']=features.spring_sales_x/(features.fall_sales_x+features.spring_sales_x+features.summer_sales_x+features.winter_sales_x)

In [273]:
features['sales_summer_rate']=features.summer_sales_x/(features.fall_sales_x+features.spring_sales_x+features.summer_sales_x+features.winter_sales_x)

In [274]:
features['sales_winter_rate']=features.winter_sales_x/(features.fall_sales_x+features.spring_sales_x+features.summer_sales_x+features.winter_sales_x)

In [275]:
features['refund_fall_rate']=features.fall_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [276]:
features['refund_fall_rate']=features.fall_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [277]:
features['refund_spring_rate']=features.spring_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [278]:
features['refund_summer_rate']=features.summer_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [279]:
features['refund_winter_rate']=features.winter_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [280]:
features['all_fall_rate']=features.fall_sales/(features.fall_sales+features.spring_sales+features.summer_sales+features.winter_sales)

In [281]:
features['all_spring_rate']=features.spring_sales/(features.fall_sales+features.spring_sales+features.summer_sales+features.winter_sales)

In [282]:
features['all_summer_rate']=features.summer_sales/(features.fall_sales+features.spring_sales+features.summer_sales+features.winter_sales)

In [283]:
features['all_winter_rate']=features.winter_sales/(features.fall_sales+features.spring_sales+features.summer_sales+features.winter_sales)

In [284]:
# 주말 구매 비율
features['wd_sales_rate'] = (data.groupby('custid')['weekend'].sum().reset_index()['weekend']/features['sales_freq'])*100

월별 구매액의 합, 평균

In [285]:
month_sales_mn=data.groupby(['custid','sales_month'])['net_amt'].mean().unstack().fillna(0)

In [286]:
month_sales_mn.columns = ['mon_sale_mn'+ str(column) for column in month_sales_mn.columns]
month_sales_mn = month_sales_mn.reset_index()

In [287]:
month_sales_sm=data.groupby(['custid','sales_month'])['net_amt'].sum().unstack().fillna(0)

In [288]:
month_sales_sm.columns = ['mon_sale_sm'+ str(column) for column in month_sales_sm.columns]
month_sales_sm = month_sales_sm.reset_index()

In [289]:
features = pd.merge(features, month_sales_sm, how='left',on = 'custid')

In [290]:
features = pd.merge(features, month_sales_mn, how='left',on = 'custid')

In [291]:
# 구매 월 평균
features['month_mean'] = data.groupby('custid')['sales_month'].mean().reset_index()['sales_month']

In [292]:
# 자주 가는 월 - 원래 자주가는 
#f = pd.pivot_table(data, index='custid', columns='sales_month', values='tot_amt', 
#                   aggfunc=np.size, fill_value=0).reset_index()
f=pd.pivot_table(data[data.tot_amt>0], index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()

In [293]:
f.columns=['custid', '1_mmff', '2_mmff', '3_mmff', '4_mmff', '5_mmff', '6_mmff', '7_mmff', '8_mmff', '9_mmff', '10_mmff', '11_mmff', '12_mmff']

In [294]:
features = pd.merge(features, f, how='left',on = 'custid')

금액 관리

In [295]:
refund_net_sum=data[data.net_amt<0].groupby('custid')['net_amt'].sum().reset_index()
features = pd.merge(features, refund_net_sum, how='left',on = 'custid')

In [296]:
net_saels_all=data.groupby('custid')['net_amt'].agg([('tot_sales_all','sum'),('mean_amt_all','mean')])
features = pd.merge(features, net_saels_all, how='left',on = 'custid').fillna(0)

In [297]:
net_sales_0_all=data[data.net_amt>0].groupby('custid')['net_amt'].agg([('tot_sales','sum'),('max_sales_amt','max'),('min_sales_amt','min'),('mean_sales_amt','mean')])
features = pd.merge(features, net_sales_0_all, how='left',on = 'custid').fillna(0)

In [298]:
net_refund_all=data[data.net_amt<0].groupby('custid')['net_amt'].agg([('tot_refund','sum'),('max_refund_amt','max'),('min_refund_amt','min'),('mean_refund_amt','mean')])
features = pd.merge(features, net_refund_all, how='left',on = 'custid').fillna(0)

In [299]:
features[['mean_amt_all','mean_sales_amt','mean_refund_amt']] =round(features[['mean_amt_all','mean_sales_amt','mean_refund_amt']],2)

In [300]:
features['tot_sales_freq_mean']=features['tot_sales']/features['sales_dayofweek']

In [301]:
# 총 할인금액
features['tot_dis_amt'] = data.groupby('custid')['dis_amt'].sum().reset_index()['dis_amt']

In [302]:
# 최대, 평균 할인율
features['max_dis'] = data.groupby('custid')['dis_rate'].max().reset_index()['dis_rate']
features['mean_dis'] = data.groupby('custid')['dis_rate'].mean().reset_index()['dis_rate']

In [303]:
# 할인 받은 빈도, 비율
dis_rate=data.query("dis_rate>0").groupby('custid')['dis_rate'].count().reset_index()
features = pd.merge(features, dis_rate, how='left',on='custid').fillna(0)
features.rename(columns = {'dis_rate': 'dis_freq'},inplace=True)

In [304]:
features['dis_freq_rate'] = features['dis_freq']/features['sales_freq']

In [305]:
# 할부 빈도, 비율/ 평균 할부개월
inst_ft = data.query("inst_mon != 1").groupby('custid')['inst_mon'].agg([('inst_freq','count'),('inst_avg','mean'),('inst_max','max')])
features = pd.merge(features, inst_ft, how='left',on = 'custid').fillna(0)
features['inst_freq_rate'] = features['inst_freq']/features['sales_freq']
features['inst_freq_all_rate'] = features['inst_freq']/features['all_freq']

In [306]:
# 할부 총 달
inst_ft_sum = data.groupby('custid')['inst_mon'].agg([('inst_sum','sum')])
features = pd.merge(features, inst_ft_sum, how='left',on = 'custid').fillna(0)

In [307]:
features['tot_inst_amt']=features['tot_sales']/features['inst_sum']

In [308]:
# 할부 총 달
sales_noinst = data.groupby('custid')['inst_fee'].agg([('inst_fee_sum','sum')])
features = pd.merge(features, sales_noinst, how='left',on = 'custid').fillna(0)

In [309]:
#def inst_avg(x):
#    if x >=3:
#        return 1
#    else:
#        return 0
#
#features['inst_quan'] = features.inst_avg.apply(inst_avg)

In [310]:
# 자주가는 지점이 어딘지 원핫인코딩 한 것
data['str_nm'] = data['str_nm'].apply(lambda x: 'mu' if x=='무역점' else('bon' if x=='본점' else('cheon' if x=='천호점' else 'sin')))

In [311]:
f = data.groupby(['custid','str_nm'])['str_nm'].agg([('str_freq','count')]).reset_index()

In [312]:
lst = []
for i in tqdm_notebook(f['custid'].unique()) : 
    f2 = f.query("custid == @i")
    lst.append(f2.sort_values(by='str_freq',ascending=False)['str_nm'].iloc[0])

features['str_freq'] = lst

  0%|          | 0/35967 [00:00<?, ?it/s]

In [313]:
features = pd.get_dummies(features, 'str_freq')

아이디 별 매장별 몇번씩 갔는지

In [314]:
str_freq_all=data.groupby(['custid','str_nm'])['str_nm'].count().unstack().fillna(0)

In [315]:
str_freq_all.columns = ['store_'+ str(column) for column in str_freq_all.columns]
str_freq_all = str_freq_all.reset_index()

In [316]:
features = pd.merge(features, str_freq_all, how='left',on = 'custid')

매장 별로 얼마나 구매를 하였는지 총구매액

In [317]:
str_sale_sum=data[data.net_amt>0].groupby(['custid','str_nm'])['net_amt'].sum().unstack().fillna(0)

In [318]:
str_sale_sum.columns = ['str_sales'+ str(column) for column in str_sale_sum.columns]
str_sale_sum = str_sale_sum.reset_index()

In [319]:
features = pd.merge(features, str_sale_sum, how='left',on = 'custid')

In [320]:
str_refund_sum=data[data.net_amt<0].groupby(['custid','str_nm'])['net_amt'].sum().unstack().fillna(0)

In [321]:
str_refund_sum.columns = ['str_refund'+ str(column) for column in str_refund_sum.columns]
str_refund_sum = str_refund_sum.reset_index()

In [322]:
features = pd.merge(features, str_refund_sum, how='left',on = 'custid')

In [323]:
# 선호 브랜드
df = pd.merge(trn, y, on='custid')

In [324]:
twn_prefer_brd = df.query("age>=20 & age<30").brd_nm.value_counts().index[1:11].tolist()
thr_prefer_brd = df.query("age>=30 & age<40").brd_nm.value_counts().index[1:11].tolist()
for_prefer_brd = df.query("age>=40 & age<50").brd_nm.value_counts().index[1:11].tolist()
fiv_prefer_brd = df.query("age>=50 & age<60").brd_nm.value_counts().index[1:11].tolist()
six_prefer_brd = df.query("age>=60").brd_nm.value_counts().index[1:11].tolist()

In [325]:
def map_twn_brd(x):
    if x == twn_prefer_brd[0]:
        x=10
    elif x == twn_prefer_brd[1]:
        x=9
    elif x == twn_prefer_brd[2]:
        x=8
    elif x == twn_prefer_brd[3]:
        x=7
    elif x == twn_prefer_brd[4]:
        x=6
    elif x == twn_prefer_brd[5]:
        x=5
    elif x == twn_prefer_brd[6]:
        x=4
    elif x == twn_prefer_brd[7]:
        x=3
    elif x == twn_prefer_brd[8]:
        x=2
    elif x == twn_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [326]:
def map_thr_brd(x):
    if x == thr_prefer_brd[0]:
        x=10
    elif x == thr_prefer_brd[1]:
        x=9
    elif x == thr_prefer_brd[2]:
        x=8
    elif x == thr_prefer_brd[3]:
        x=7
    elif x == thr_prefer_brd[4]:
        x=6
    elif x == thr_prefer_brd[5]:
        x=5
    elif x == thr_prefer_brd[6]:
        x=4
    elif x == thr_prefer_brd[7]:
        x=3
    elif x == thr_prefer_brd[8]:
        x=2
    elif x == thr_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [327]:
def map_for_brd(x):
    if x == for_prefer_brd[0]:
        x=10
    elif x == for_prefer_brd[1]:
        x=9
    elif x == for_prefer_brd[2]:
        x=8
    elif x == for_prefer_brd[3]:
        x=7
    elif x == for_prefer_brd[4]:
        x=6
    elif x == for_prefer_brd[5]:
        x=5
    elif x == for_prefer_brd[6]:
        x=4
    elif x == for_prefer_brd[7]:
        x=3
    elif x == for_prefer_brd[8]:
        x=2
    elif x == for_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [328]:
def map_fiv_brd(x):
    if x == fiv_prefer_brd[0]:
        x=10
    elif x == fiv_prefer_brd[1]:
        x=9
    elif x == fiv_prefer_brd[2]:
        x=8
    elif x == fiv_prefer_brd[3]:
        x=7
    elif x == fiv_prefer_brd[4]:
        x=6
    elif x == fiv_prefer_brd[5]:
        x=5
    elif x == fiv_prefer_brd[6]:
        x=4
    elif x == fiv_prefer_brd[7]:
        x=3
    elif x == fiv_prefer_brd[8]:
        x=2
    elif x == fiv_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [329]:
def map_six_brd(x):
    if x == six_prefer_brd[0]:
        x=10
    elif x == six_prefer_brd[1]:
        x=9
    elif x == six_prefer_brd[2]:
        x=8
    elif x == six_prefer_brd[3]:
        x=7
    elif x == six_prefer_brd[4]:
        x=6
    elif x == six_prefer_brd[5]:
        x=5
    elif x == six_prefer_brd[6]:
        x=4
    elif x == six_prefer_brd[7]:
        x=3
    elif x == six_prefer_brd[8]:
        x=2
    elif x == six_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [330]:
data['twn_brd_point'] = data['brd_nm'].apply(map_twn_brd)
data['thr_brd_point'] = data['brd_nm'].apply(map_thr_brd)
data['for_brd_point'] = data['brd_nm'].apply(map_for_brd)
data['fiv_brd_point'] = data['brd_nm'].apply(map_fiv_brd)
data['six_brd_point'] = data['brd_nm'].apply(map_six_brd)

In [331]:
twn_brd_point=data.groupby('custid')['twn_brd_point'].mean().reset_index()
thr_brd_point=data.groupby('custid')['thr_brd_point'].mean().reset_index()
for_brd_point=data.groupby('custid')['for_brd_point'].mean().reset_index()
fiv_brd_point=data.groupby('custid')['fiv_brd_point'].mean().reset_index()
six_brd_point=data.groupby('custid')['six_brd_point'].mean().reset_index()

In [332]:
features = pd.merge(features, twn_brd_point, how='left',on='custid')
features = pd.merge(features, thr_brd_point, how='left',on='custid')
features = pd.merge(features, for_brd_point, how='left',on='custid')
features = pd.merge(features, fiv_brd_point, how='left',on='custid')
features = pd.merge(features, six_brd_point, how='left',on='custid')

In [333]:
data['buyer_nm'] = data['buyer_nm'].str.replace('행사장.*','행사장')
data['buyer_nm'] = data['buyer_nm'].str.replace('피혁A','피혁')
data['buyer_nm'] = data['buyer_nm'].str.replace('피혁B','피혁')

data['buyer_nm'] = data['buyer_nm'].map({'화장품':'a', '수입명품':'b', '캐릭터캐주얼':'c', '트래디셔널캐주얼':'d', '유아동복':'e', '니트단품':'f', '영캐주얼':'g',
       '엘레강스캐주얼':'h', '가전':'i', '섬유':'j', '장신구':'k', '조리욕실':'l', '스포츠':'m', '침구수예':'n', '피혁':'o', '일반식품':'p',
       '유니캐주얼':'q', '정장셔츠':'r', '디자이너부띠끄':'s', '문화완구':'t', '타운모피':'u', '조리식품':'v', '기타바이어':'w',
       '도자기크리스탈':'x', '가구':'y', '생활용품':'z', '행사장':'aa', '청과곡물':'bb', '점외':'cc'})

buyer = pd.pivot_table(index='custid',columns='buyer_nm',values='tot_amt',aggfunc=np.size,fill_value=0,data=data.query("tot_amt>0")[['custid','buyer_nm','tot_amt']]).reset_index()
features = pd.merge(features, buyer, on='custid')

In [334]:
#구매상품군 다양성
n = data.part_nm.nunique()
f = data.groupby('custid')['part_nm'].agg([('part_unique', lambda x: len(x.unique()) / n)]).reset_index()
features = pd.merge(features, f, on='custid')

In [335]:
#브랜드 다양성
n=data.brd_nm.nunique()
f = data.groupby('custid')['brd_nm'].agg([('brd_unique', lambda x: len(x.unique()) / n)]).reset_index()
features = pd.merge(features, f, on='custid')

In [336]:
#매장 이용 다양성
n = 4
f = data.groupby('custid')['str_nm'].agg([('str_unique', lambda x: len(x.unique()) / n)]).reset_index()
features = pd.merge(features, f, on='custid')

In [337]:
#위에서는 출근 시간 9~17시 퇴근시간 이후로 했는데 좀더 세분화
def f2(x):
    if 9 <= x < 12 :
        return('9_12_sale')
    elif 12 <= x < 14 :
        return('12_14_sale')
    elif 14 <= x < 16 :
        return('14_16_sale')
    elif 16 <= x < 18 :
        return('16_18_sale')
    else :
        return('18_sale')  

data['timeslot2'] = data.sales_time.apply(f2)

In [338]:
new_sales_type = pd.pivot_table(index='custid',columns='timeslot2',values='tot_amt',aggfunc=np.size, fill_value=0,data = data.query('tot_amt>0')[['custid','timeslot2','tot_amt']]).reset_index()

In [339]:
features=pd.merge(features, new_sales_type, on='custid')

In [340]:
features['12_14_sale_ratio']=features['12_14_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])
features['14_16_sale_ratio']=features['14_16_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])
features['16_18_sale_ratio']=features['16_18_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])
features['18_sale_ratio']=features['18_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])
features['9_12_sale_ratio']=features['9_12_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])

In [341]:
#충동지수 충동구매를 하는사람들은 누구일까? 여유가있는 나이가 많은 사람들이 충동구매를 할거같다. 곱하기대신 더하기도 생각해보기

features['chungdong_sale']=(features.max_sales_amt/features.tot_sales)
features['chungdong_sale_freq']= features.chungdong_sale*features.sales_freq

In [342]:
features['mean_min_sales']=features.mean_sales_amt+features.min_sales_amt

In [343]:
a=features[['custid','mean_min_sales']]

In [344]:
df_LEFT_JOIN = pd.merge(data, a, left_on='custid', right_on='custid', how='left')

In [345]:
df_LEFT_JOIN['chungdong']=df_LEFT_JOIN.net_amt >df_LEFT_JOIN.mean_min_sales

In [346]:
chung=df_LEFT_JOIN.groupby('custid')['chungdong'].sum().reset_index()

In [347]:
features = pd.merge(features, chung, left_on='custid', right_on='custid', how='left')

In [348]:
a=data.groupby(['custid','sales_month','sales_day','sales_dayofweek'])['custid'].size().unstack().reset_index()

In [349]:
a.iloc[:,3:]=a.iloc[:,3:]/a.iloc[:,3:]

In [350]:
a=a.groupby('custid')[['금','목','수','월','일','토','화']].sum()

In [351]:
features = pd.merge(features, a, on='custid')

In [352]:
features = features.rename(columns={'월':'mon_opn','화':'tue_opn','수':'wen_opn','목':'thr_opn','금':'fri_opn','토':'sat_opn','일':'sun_opn'})

In [353]:
features['mon_opn_rate']=features.mon_opn/features.sales_dayofweek
features['tue_opn_rate']=features.tue_opn/features.sales_dayofweek
features['wen_opn_rate']=features.wen_opn/features.sales_dayofweek
features['thr_opn_rate']=features.thr_opn/features.sales_dayofweek
features['fri_opn_rate']=features.fri_opn/features.sales_dayofweek
features['sat_opn_rate']=features.sat_opn/features.sales_dayofweek
features['sun_opn_rate']=features.sun_opn/features.sales_dayofweek

In [354]:
a=data[data.net_amt>0].groupby(['custid','sales_month','sales_day','sales_dayofweek'])['net_amt'].sum().unstack().reset_index()

In [355]:
a=a.groupby('custid')[['금','목','수','월','일','토','화']].sum()

In [356]:
features = pd.merge(features, a, on='custid')

In [357]:
features = features.rename(columns={'월':'mon_pri','화':'tue_pri','수':'wen_pri','목':'thr_pri','금':'fri_pri','토':'sat_pri','일':'sun_pri'})

In [358]:
features['mon_pri_rate']=features.mon_pri/features.tot_sales
features['tue_pri_rate']=features.tue_pri/features.tot_sales
features['wen_pri_rate']=features.wen_pri/features.tot_sales
features['thr_pri_rate']=features.thr_pri/features.tot_sales
features['fri_pri_rate']=features.fri_pri/features.tot_sales
features['sat_pri_rate']=features.sat_pri/features.tot_sales
features['sun_pri_rate']=features.sun_pri/features.tot_sales

충동구매를 하고 환불을 한 횟수

In [359]:
features['max_min_sales']=features['max_sales_amt']-features['min_sales_amt']

In [360]:
def shopfnc(m):
    if m < 5:
        return 1
    elif m < 12.94 :
        return 2 
    elif m < 21.538 :
        return 3 
    elif m < 33.909 :
        return 4
    else:
        return 5

In [361]:
features['shop_time_mean']= features.shop_time.apply(lambda x : shopfnc(x))

In [362]:
for i in np.arange(0.1,1,0.1):
    print(features.tot_sales.quantile(q=i))

249406.0
498646.0000000001
795650.0
1165370.0000000002
1622598.0
2254104.0
3107333.5999999996
4541631.800000001
7485838.200000001


In [363]:
# 10분위별로 (1 ~ 10 범위)
def fnc(m) : 
    if m < 249406.0:
        return 1
    elif m <498646 :
        return 2 
    elif m < 795650.0 :
        return 3 
    elif m < 1165370:
        return 4 
    elif m < 1622598.0 :
        return 5 
    elif m < 2254104 :
        return 6 
    elif m < 3107333:
        return 7 
    elif m < 4541631 :
        return 8
    elif m < 7485838 :
        return 9 
    else:
        return 10

features['tot_sales_amt_quan']= features.tot_sales.apply(lambda x : fnc(x))

In [364]:
for i in np.arange(0.1,1,0.1):
    print(features.max_sales_amt.quantile(q=i))

98400.0
149000.0
199000.0
261250.0
330600.0
416000.0
534850.0
736250.0
1180000.0


In [365]:
# 10분위별로 가격선호도 조사 (1 ~ 10 범위)
def fnc2(m) : 
    if m < 98400.0 :
        return 1
    elif m < 149000.0 :
        return 2 
    elif m < 199000.0 :
        return 3 
    elif m < 261250.0 :
        return 4 
    elif m < 330600.0 :
        return 5 
    elif m < 416000.0 :
        return 6 
    elif m < 534850.0 :
        return 7 
    elif m < 736250.0 :
        return 8
    elif m < 1180000.0 :
        return 9 
    else:
        return 10

features['max_amt_quan']= features.max_sales_amt.apply(lambda x : fnc2(x))

In [366]:
#아동상품 구매건수
f = data[data.tot_amt > 0].groupby('custid')['part_nm'].agg([('baby_sales', lambda x: list(x).count('아동')+list(x).count('케주얼,구두,아동')+list(x).count('아동문화')+list(x).count('아동,스포츠'))]).reset_index()
features = pd.merge(features, f, on='custid')

In [367]:
features['baby_sales_freq']=features['baby_sales']/features.sales_freq

In [368]:
#할부 총합
sales_installment=data.groupby('custid')['inst_mon'].agg('sum').reset_index()
features = pd.merge(features, sales_installment, on='custid')

In [369]:
data['team_nm'] = data['team_nm'].map({'잡화가용팀':'aa', '의류패션팀':'bb', '식품팀':'cb', '인터넷백화점':'dd'})

In [370]:
f = data.groupby(['custid', 'team_nm'])['custid'].count().unstack()
f = f.fillna(0)
f.columns = ['Team_'+ str(column) for column in f.columns]
f = f.reset_index()

In [371]:
features = pd.merge(features, f, on = 'custid')

In [372]:
features['food_rate']=features.Team_cb/(features.Team_aa+features.Team_bb+features.Team_cb+features.Team_dd)

In [373]:
features['cloth_rate']=features.Team_bb/(features.Team_aa+features.Team_bb+features.Team_cb+features.Team_dd)

In [374]:
features['stuff_rate']=features.Team_aa/(features.Team_aa+features.Team_bb+features.Team_cb+features.Team_dd)

In [375]:
food_list=['조리식품','축산물','농산물','수산물']
t1 = data[data.net_amt>0].query("pc_nm == @food_list").groupby('custid')['net_amt'].agg([('food_sales', np.sum)])
features= pd.merge(features,t1, on = 'custid', how='left').fillna(0)
features['food_sales_rate'] = features['food_sales'] / features['tot_sales']

In [376]:
st=f"(net_amt>0) and (pc_nm=='화장품')"

In [377]:
t2 = data.query(st).groupby('custid')['net_amt'].agg([('face_goods_sales', np.sum)])
features= pd.merge(features,t2, on = 'custid', how='left').fillna(0)
features['face_sales_rate'] = features['face_goods_sales'] / features['tot_sales']

In [378]:
t2 = data.query(st).groupby('custid')['net_amt'].agg([('face_goods_freq', np.size)])
features= pd.merge(features,t2, on = 'custid', how='left').fillna(0)
features['face_goods_rate'] = features['face_goods_freq'] / features['sales_freq']

In [379]:
data['pc_nm'] = data['pc_nm'].str.replace('디자이너부띠끄','디자이너부띠크')
data['pc_nm'] = data['pc_nm'].str.replace('디자이너부틱','디자이너부띠크')
data['pc_nm'] = data['pc_nm'].str.replace('트래디셔널','트레디셔널')
data['pc_nm'] = data['pc_nm'].str.replace('침구/수예','침구수예')
data['pc_nm'] = data['pc_nm'].str.replace('침구,수예','침구수예')
data['pc_nm'] = data['pc_nm'].str.replace('피혁A','피혁')
data['pc_nm'] = data['pc_nm'].str.replace('피혁B','피혁')
data['pc_nm'] = data['pc_nm'].str.replace('캐릭터케쥬얼','캐릭터캐쥬얼')
data['pc_nm'] = data['pc_nm'].str.replace('캐릭터캐주얼','캐릭터캐쥬얼')
data['pc_nm'] = data['pc_nm'].str.replace('진캐주얼','진캐쥬얼')
data['pc_nm'] = data['pc_nm'].str.replace('진케주얼','진캐쥬얼')
data['pc_nm'] = data['pc_nm'].str.replace('로얄부틱','로얄부띠크')
data['pc_nm'] = data['pc_nm'].str.replace('로얄부띠끄','로얄부띠크')

In [380]:
twn_trend_brd = df.query("age>=20 & age<30").pc_nm.value_counts().index[1:11].tolist()
thr_trend_brd = df.query("age>=30 & age<40").pc_nm.value_counts().index[1:11].tolist()
for_trend_brd = df.query("age>=40 & age<50").pc_nm.value_counts().index[1:11].tolist()
fiv_trend_brd = df.query("age>=50 & age<60").pc_nm.value_counts().index[1:11].tolist()
six_trend_brd = df.query("age>=60").pc_nm.value_counts().index[1:11].tolist()

def map_twn_pc(x):
    if x == twn_trend_brd[0]:
        x=10
    elif x == twn_trend_brd[1]:
        x=9
    elif x == twn_trend_brd[2]:
        x=8
    elif x == twn_trend_brd[3]:
        x=7
    elif x == twn_trend_brd[4]:
        x=6
    elif x == twn_trend_brd[5]:
        x=5
    elif x == twn_trend_brd[6]:
        x=4
    elif x == twn_trend_brd[7]:
        x=3
    elif x == twn_trend_brd[8]:
        x=2
    elif x == twn_trend_brd[9]:
        x=1
    else:
        x=0
    return x

def map_thr_pc(x):
    if x == thr_trend_brd[0]:
        x=10
    elif x == thr_trend_brd[1]:
        x=9
    elif x == thr_trend_brd[2]:
        x=8
    elif x == thr_trend_brd[3]:
        x=7
    elif x == thr_trend_brd[4]:
        x=6
    elif x == thr_trend_brd[5]:
        x=5
    elif x == thr_trend_brd[6]:
        x=4
    elif x == thr_trend_brd[7]:
        x=3
    elif x == thr_trend_brd[8]:
        x=2
    elif x == thr_trend_brd[9]:
        x=1
    else:
        x=0
    return x

def map_for_pc(x):
    if x == for_trend_brd[0]:
        x=10
    elif x == for_trend_brd[1]:
        x=9
    elif x == for_trend_brd[2]:
        x=8
    elif x == for_trend_brd[3]:
        x=7
    elif x == for_trend_brd[4]:
        x=6
    elif x == for_trend_brd[5]:
        x=5
    elif x == for_trend_brd[6]:
        x=4
    elif x == for_trend_brd[7]:
        x=3
    elif x == for_trend_brd[8]:
        x=2
    elif x == for_trend_brd[9]:
        x=1
    else:
        x=0
    return x

def map_fiv_pc(x):
    if x == fiv_trend_brd[0]:
        x=10
    elif x == fiv_trend_brd[1]:
        x=9
    elif x == fiv_trend_brd[2]:
        x=8
    elif x == fiv_trend_brd[3]:
        x=7
    elif x == fiv_trend_brd[4]:
        x=6
    elif x == fiv_trend_brd[5]:
        x=5
    elif x == fiv_trend_brd[6]:
        x=4
    elif x == fiv_trend_brd[7]:
        x=3
    elif x == fiv_trend_brd[8]:
        x=2
    elif x == fiv_trend_brd[9]:
        x=1
    else:
        x=0
    return x

def map_six_pc(x):
    if x == six_trend_brd[0]:
        x=10
    elif x == six_trend_brd[1]:
        x=9
    elif x == six_trend_brd[2]:
        x=8
    elif x == six_trend_brd[3]:
        x=7
    elif x == six_trend_brd[4]:
        x=6
    elif x == six_trend_brd[5]:
        x=5
    elif x == six_trend_brd[6]:
        x=4
    elif x == six_trend_brd[7]:
        x=3
    elif x == six_trend_brd[8]:
        x=2
    elif x == six_trend_brd[9]:
        x=1
    else:
        x=0
    return x

data['twn_pc_point'] = data['pc_nm'].apply(map_twn_pc)
data['thr_pc_point'] = data['pc_nm'].apply(map_thr_pc)
data['for_pc_point'] = data['pc_nm'].apply(map_for_pc)
data['fiv_pc_point'] = data['pc_nm'].apply(map_fiv_pc)
data['six_pc_point'] = data['pc_nm'].apply(map_six_pc)

twn_pcnm_point=data.groupby('custid')['twn_pc_point'].mean().reset_index()
thr_pcnm_point=data.groupby('custid')['thr_pc_point'].mean().reset_index()
for_pcnm_point=data.groupby('custid')['for_pc_point'].mean().reset_index()
fiv_pcnm_point=data.groupby('custid')['fiv_pc_point'].mean().reset_index()
six_pcnm_point=data.groupby('custid')['six_pc_point'].mean().reset_index()

features = pd.merge(features, twn_pcnm_point, how='left',on='custid')
features = pd.merge(features, thr_pcnm_point, how='left',on='custid')
features = pd.merge(features, for_pcnm_point, how='left',on='custid')
features = pd.merge(features, fiv_pcnm_point, how='left',on='custid')
features = pd.merge(features, six_pcnm_point, how='left',on='custid')

In [381]:
f = data[data.tot_amt > 0].groupby('custid')['pc_nm'].agg([('young_pc', lambda x: list(x).count('캐릭터정장')+list(x).count('영트랜디')+list(x).count('영캐릭터캐쥬얼')+list(x).count('트랜디 케쥬얼')+list(x).count('캐릭터캐쥬얼')+list(x).count('장신구')+list(x).count('피혁')+list(x).count('섬유')+list(x).count('소품')+list(x).count('핸드백')+list(x).count('(주)현스포츠아쌤'))]).reset_index()
features = pd.merge(features, f, on='custid')

In [382]:
features['20goods_rate']=features['young_pc']/features.sales_freq

In [383]:
#50대선호

f = data[data.tot_amt > 0].groupby('custid')['pc_nm'].agg([('butique_freq', lambda x: list(x).count('로얄부띠크')+list(x).count('디자이너부띠크'))]).reset_index()
features = pd.merge(features, f, on='custid')

In [384]:
features['50goods_rate']=features['butique_freq']/features.sales_freq

In [385]:
#트랜드 명품 구매비율
f = data[data.tot_amt > 0].groupby('custid')['pc_nm'].agg([('expensive_freq', lambda x: list(x).count('명품토탈')+list(x).count('명품시즌'))]).reset_index()
features = pd.merge(features, f, on='custid')

In [386]:
features['20~30expensive_rate']=features['expensive_freq']/features.sales_freq

In [387]:
data['pc_nm'] = data['pc_nm'].map({'화장품':'face_goods', '명품토탈':'expensive_total', '캐릭터정장':'chr_suit',  '수예':'suae', '유아/완구':'baby_wangu', '패션슈즈':'fashoin_shoes',
       '가전':'electronic', '영트랜디':'young_trend', '조리식품':'instant_food', '스포츠':'sports', '아동':'baby', '트래디셔널캐쥬얼':'trady_ca', '가전/문화':'ele_cultur', '셔츠':'shirts',
       '란제리':'rangeri', '식기':'food_case', '영캐릭터캐쥬얼':'young_chr', '골프':'golf', '캐릭터캐쥬얼':'chr_cha', '엘레강스캐쥬얼':'elegangs', '트레디셔널':'tradi',
       '정장':'suit', '일반식품':'normal_food', '아동복':'baby_cloth', '축산물':'pig', '미확인pc':'unknown_pc','농산물':'vegetable','트랜디 케쥬얼':'trendi cha',
        '영트렌디':'young_trendi','여성캐주얼':'girl_cha','문화용품':'culture_goods','생활용품':'lsang','문화':'culture','니트,단품,모피':'neat_mofi','섬유잡화':'sumyou_goods','내의란제리':'inrangeri',
        '진캐쥬얼':'jean_ca3','여성구두':'girl_shoes','명품시즌':'expensive_season','남여구두':'man_girl','장신구':'jangsingu','수산물':'susanmul','악세사리':'accesory','섬유':'sumyou',
        '어덜트캐주얼':'adult_cha','소품':'sopeum','캐주얼':'cha','미씨캐주얼':'messi_cha','영캐주얼':'young_cha','타운란제리':'townrangeri',
        '핸드백':'handbag','수입명품':'suip_goods','디자이너부띠크':'designier_beatuty','쇼핑보증':'shooping_bo','미씨캐릭터':'messi_cha5',
        '가구':'gagu','캐쥬얼':'cha2','니트/단품':'neat_one','남성잡화':'man_jobs','(주)현스포츠아쌤':'sportsam','침구수예':'chimgu4','패션시즌':'fashion_season',
        '수입의류':'suip_cloth','용기보증':'youngi','사이버쇼핑':'cyber_shopping'})

In [388]:
#주구매상품
f = data.groupby('custid')['pc_nm'].agg([
    ('main_goods', lambda x: x.value_counts().index[0])
]).reset_index()

In [389]:
features = features.merge(f, how='left')
features = pd.get_dummies(features, 'main_goods')

In [390]:
f_p = data[data.tot_amt>0].groupby('pc_nm')['tot_amt'].agg([
    ('main_goods_price', 'mean')
]).reset_index()

In [391]:
f = pd.merge(f, f_p,  left_on = 'main_goods',right_on='pc_nm',how='left')

In [392]:
features = pd.merge(features, f[['custid','main_goods_price']],  left_on = 'custid',right_on='custid',how='left')

In [393]:
pc_count=data[data.tot_amt>0].groupby(['custid','pc_nm'])['pc_nm'].count().unstack().fillna(0)

pc_count.columns = ['pc_size_'+ str(column) for column in pc_count.columns]
pc_count = pc_count.reset_index()

features = pd.merge(features, pc_count, how='left',on = 'custid')

In [394]:

data['part_nm'] = data['part_nm'].map({'명품잡화':'aaa', '잡화파트':'bbb', 
                                       '남성의류':'ccc', '골프/유니캐쥬얼':'ddd', 
                                       '가정용품':'eee', '아동':'fff', '잡화':'ggg',
                                       '영어덜트캐쥬얼':'hhh', '영라이브':'iii',
        '공산품':'jjj', '스포츠캐쥬얼':'kkk', '여성캐주얼':'lll', '여성정장':'mmm',
            '케주얼,구두,아동':'nnn', '남성정장스포츠':'ooo', '아동문화':'ppp',
'로얄부띠끄':'qqq', '공산품파트':'rrr', '생식품파트':'sss', '여성의류파트':'ttt',
                '가정용품파트':'uuu', '아동,스포츠':'vvv', '패션잡화':'www',
       '영캐릭터':'abbb', '영플라자':'abbc', '생식품':'abbd', '스포츠캐주얼':'abbe',
                '여성캐쥬얼':'abbf', '로얄부틱':'abbg','인터넷백화점':'abbh'})


In [395]:
data.part_nm

0         aaa
1         aaa
2         bbb
3         bbb
4         aaa
         ... 
414950    ppp
414951    www
414952    nnn
414953    nnn
414954    aaa
Name: part_nm, Length: 1040039, dtype: object

In [396]:
#주구매상품분류
f = data.groupby('custid')['part_nm'].agg([
    ('main_part', lambda x: x.value_counts().index[0])
]).reset_index()

In [397]:
features = features.merge(f, how='left')
features = pd.get_dummies(features, 'main_part')

In [398]:
f_p = data[data.tot_amt>0].groupby('part_nm')['tot_amt'].agg([
    ('main_part_price', 'mean')
]).reset_index()

In [399]:
f = pd.merge(f, f_p,  left_on = 'main_part',right_on='part_nm',how='left')

In [400]:
features = pd.merge(features, f[['custid','main_part_price']],  left_on = 'custid',right_on='custid',how='left')

In [401]:
part_count=data[data.tot_amt>0].groupby(['custid','part_nm'])['part_nm'].count().unstack().fillna(0)

part_count.columns = ['part_size_'+ str(column) for column in part_count.columns]
part_count = part_count.reset_index()

features = pd.merge(features, part_count, how='left',on = 'custid')

In [402]:
#내점 당 구매건수 / 내점당 구매액
features['one_sales_freq']=features.sales_freq/features.sales_dayofweek
features['one_tot_amt']=features.tot_sales/features.sales_dayofweek

In [403]:
data['part_pc'] = data['part_nm'].astype(str) + "_" + data['pc_nm'].astype(str)

In [404]:
#주구매상품분류
f = data.groupby('custid')['part_pc'].agg([
    ('main_part_pc', lambda x: x.mode()[0])
]).reset_index()

In [405]:
features = features.merge(f, how='left')
features = pd.get_dummies(features, 'main_part_pc')

In [406]:
f_p = data[data.tot_amt>0].groupby('part_pc')['tot_amt'].agg([
    ('main_part_pc_price', 'mean')
]).reset_index()

In [407]:
f = pd.merge(f, f_p,  left_on = 'main_part_pc',right_on='part_pc',how='left')

In [408]:
features = pd.merge(features, f[['custid','main_part_pc_price']],  left_on = 'custid',right_on='custid',how='left')

In [409]:
part_count=data[data.tot_amt>0].groupby(['custid','part_pc'])['part_pc'].count().unstack().fillna(0)

part_count.columns = ['part_pc_size_'+ str(column) for column in part_count.columns]
part_count = part_count.reset_index()

features = pd.merge(features, part_count, how='left',on = 'custid')

In [410]:
features1=features.copy()

In [411]:
i_max=[]
for i in list(features.columns):
    if features[i].max()>50000:
        i_max.append(i)

In [412]:
i_max_min=[]
for i in i_max:
    if features[i].min()>0:
        i_max_min.append(i)

In [413]:
features[i_max_min]=np.log1p(features[i_max_min])

In [414]:
# 데이터 분할
X_train = pd.DataFrame({'custid':trn['custid'].unique()})
X_train = pd.merge(X_train, features, how='left',on='custid')

y_train = y['age']

X_test = pd.DataFrame({'custid':tst['custid'].unique()})
X_test = pd.merge(X_test, features, how='left',on='custid')

In [415]:
X_train.drop('custid',axis=1,inplace=True)
X_test.drop('custid',axis=1,inplace=True)

In [416]:
X_train.shape, X_test.shape, y_train.shape

((21587, 757), (14380, 757), (21587,))

In [417]:
X_columns=X_train.columns

In [418]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [419]:
X_train_scaled=pd.DataFrame(X_train_scaled,columns=X_columns)
X_test_scaled=pd.DataFrame(X_test_scaled,columns=X_columns)

In [421]:
X_train_scaled.shape, X_test_scaled.shape, y_train.shape

((21587, 757), (14380, 757), (21587,))

In [ ]:
X_train_nothuman=X_train_scaled.copy()

In [ ]:
X_test_nothuman=X_test_scaled.copy()

In [423]:
X_train_scaled.to_csv('2등피쳐_W2V빼고.csv')
X_test_scaled.to_csv('2등피쳐_W2V빼고_test.csv')